In [8]:
!pip install transformers

!pip install timm  # PyTorch Image Models 
!pip install pillow numpy matplotlib
!pip install opencv-python
!pip install pandas

!pip install datasets  # HuggingFace datasets library

  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
  Using cached colorama-0.4.6-py2.py3-none-any.whl.metadata (17 kB)
   ---------------------------------------- 0.0/12.0 MB ? eta -:--:--
   ------------------------- -------------- 7.6/12.0 MB 39.0 MB/s eta 0:00:01
   ---------------------------------------- 12.0/12.0 MB 46.9 MB/s  0:00:00
   ---------------------------------------- 0.0/564.3 kB ? eta -:--:--
   ---------------------------------------- 564.3/564.3 kB 18.9 MB/s  0:00:00
   ---------------------------------------- 0.0/2.7 MB ? eta -:--:--
   ---------------------------------------- 2.7/2.7 MB 52.4 MB/s  0:00:00
   ---------------------------------------- 0.0/12.9 MB ? eta -:--:--
   ---------------------------------------- 12.9/12.9 MB 67.6 MB/s  0:00:00
Using cached tqdm-4.67.1-py3-none-any.whl (78 kB)
Using cached colorama-0.4.6-py2.py3-none-any.whl (25 kB)

   ----------------------------------------  0/18 [urllib3]
   ---- ---------------------------------

ERROR: Invalid requirement: '#': Expected package name at the start of dependency specifier
    #
    ^


  Using cached cycler-0.12.1-py3-none-any.whl.metadata (3.8 kB)
  Using cached python_dateutil-2.9.0.post0-py2.py3-none-any.whl.metadata (8.4 kB)
  Using cached six-1.17.0-py2.py3-none-any.whl.metadata (1.7 kB)
   ---------------------------------------- 0.0/7.0 MB ? eta -:--:--
   ---------------------------------- ----- 6.0/7.0 MB 37.0 MB/s eta 0:00:01
   ---------------------------------------- 7.0/7.0 MB 30.8 MB/s  0:00:00
   ---------------------------------------- 0.0/8.1 MB ? eta -:--:--
   ---------------------------------------- 8.1/8.1 MB 83.5 MB/s  0:00:00
Using cached cycler-0.12.1-py3-none-any.whl (8.3 kB)
   ---------------------------------------- 0.0/2.3 MB ? eta -:--:--
   ---------------------------------------- 2.3/2.3 MB 42.9 MB/s  0:00:00
Using cached python_dateutil-2.9.0.post0-py2.py3-none-any.whl (229 kB)
Using cached six-1.17.0-py2.py3-none-any.whl (11 kB)

   -------- ------------------------------- 2/9 [pillow]
   -------- ------------------------------- 2/9 

ERROR: Invalid requirement: '#': Expected package name at the start of dependency specifier
    #
    ^


In [11]:
# test ViT model 

import torch
from transformers import ViTImageProcessor, ViTForImageClassification
from PIL import Image
import requests

print("Loading ViT model...")
processor = ViTImageProcessor.from_pretrained('google/vit-base-patch16-224')
model = ViTForImageClassification.from_pretrained('google/vit-base-patch16-224')

device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)
print(f"Using device: {device}")

url = 'http://images.cocodataset.org/val2017/000000039769.jpg'
image = Image.open(requests.get(url, stream=True).raw)

# process image
inputs = processor(images=image, return_tensors="pt").to(device)

print("\nRunning inference...")
with torch.no_grad():
    outputs = model(**inputs)
    logits = outputs.logits
    predicted_class_idx = logits.argmax(-1).item()

# get top 5 predictions
probs = torch.nn.functional.softmax(logits, dim=-1)[0]
top5_prob, top5_idx = torch.topk(probs, 5)

print("\nTop 5 predictions:")
for i in range(5):
    class_idx = top5_idx[i].item()
    prob = top5_prob[i].item()
    class_name = model.config.id2label[class_idx]
    print(f"{i+1}. {class_name}: {prob*100:.2f}%")

print("\n✓ ViT test successful!")

Loading ViT model...
Using device: cpu

Running inference...

Top 5 predictions:
1. Egyptian cat: 93.74%
2. tabby, tabby cat: 3.84%
3. tiger cat: 1.44%
4. lynx, catamount: 0.33%
5. Siamese cat, Siamese: 0.07%

✓ ViT test successful!
